In [1]:
import numpy as np # 只会用到这个
import json # 用来加载数据

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    if x <= 0:
        return 0
    return x

In [3]:
def load_data(path):
    with open(path, 'r') as f:
        json_data = json.load(f)
        X = np.array(json_data['x'])
        X = (X - np.mean(X,axis=0)) / np.std(X, axis=0)
        y = np.array(json_data['y'])
        return X,y 

def load_ckpt(ckpt_file_name:str):
    with open(ckpt_file_name, "r") as f:
        json_data = json.load(f)
        weights = json_data["weights"]
        return weights

In [4]:
class Layer:
    def __init__(self, input_dim, output_dim, function):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.function = function
        self.weight = np.random.randn(input_dim+1, output_dim)
    def forward(self, x):
        arr = []
        for vector in x:
            arr.append(np.append(vector, 1))
        x = np.array(arr)
        return self.function(x@self.weight)
    # def backward(self, delta): We'll do this next time!

In [16]:
class MLP:
    def __init__(self, shapes):
        self.layers = [
            Layer(shapes[i], shapes[i+1], sigmoid)
            for i in range(len(shapes)-1) #一层线是一个Layer
        ]
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    # def backward(self, y): We will do this next time!
    # def train(self, train_set, loss='mse', epochs=100, lr=0.3): We will do this next time!
    
    def evaluate(self, X, y):
        pred = self.forward(X)
        total_correct = 0
        for i in range(len(pred)):
            predi = pred[i][0]
            if np.abs(predi - y[i]) < 0.5:
                total_correct += 1
        accuracy = total_correct / len(pred)
        return accuracy

In [25]:
def main():
    mlp = MLP([2, 64, 1]) #shapes: 各层节点数
    test_X, test_Y = load_data('data8.json')
    weights = load_ckpt('quad8.ckpt')
    for i in range(len(mlp.layers)):
        mlp.layers[i].weight = np.array(weights[i]).T
    print(mlp.evaluate(test_X, test_Y))

In [29]:
main()

0.954
